<a href="https://colab.research.google.com/github/ImaduddinAhmedMohammed/ImaduddinAhmed_INFO5731_Spring2024/blob/main/Mohammed_Imad_5731_Exercise_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 In-class Exercise 5**

**This exercise aims to provide a comprehensive learning experience in text analysis and machine learning techniques, focusing on both text classification and clustering tasks.**

***Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks***.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission, and no requests will be answered. Manage your time accordingly.**


## **Question 1 (20 Points)**

The purpose of the question is to practice different machine learning algorithms for **text classification** as well as the performance evaluation. In addition, you are requried to conduct **10 fold cross validation** (https://scikit-learn.org/stable/modules/cross_validation.html) in the training.



The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data.


**Algorithms:**

*   MultinominalNB
*   SVM
*   KNN
*   Decision tree
*   Random Forest
*   XGBoost
*   Word2Vec
*   BERT

**Evaluation measurement:**


*   Accuracy
*   Recall
*   Precison
*   F-1 score


In [20]:
# Write your code here
import pandas as pd
train_data = pd.read_csv('stsa-train.txt',sep = 'delimiter=',header= None,names=['text'])
train_data[['sentiment', 'text']] = train_data['text'].str.split(" ", n=1, expand=True)
train_data.head()

<ipython-input-20-732bf6eca74b>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_data = pd.read_csv('stsa-train.txt',sep = 'delimiter=',header= None,names=['text'])


,text,sentiment
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [21]:
test_data = pd.read_csv('stsa-test.txt',sep = 'delimiter=',header= None,names=['text'])
test_data[['sentiment','text']] = test_data['text'].str.split(" ", n=1, expand=True)
test_data.head()

<ipython-input-21-948ab59e9dc1>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_data = pd.read_csv('stsa-test.txt',sep = 'delimiter=',header= None,names=['text'])


,text,sentiment
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1


In [22]:
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean(review):
    review ="".join([word.lower() for word in review if word not in string.punctuation])
    review = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", review)
    tokens = re.split('\W+',review)
    review = [wl.lemmatize(word) for word in tokens if word not in stopword]
    return review

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean)
X_tfidf = tfidf_vect.fit_transform(train_data['text'])

print(X_tfidf.shape)



X_tfidf_df=pd.DataFrame(X_tfidf.toarray())
X_tfidf_df.columns = tfidf_vect.get_feature_names_out()
X_test_tfidf = tfidf_vect.transform(test_data['text'])
print(X_test_tfidf.shape)

(6920, 13343)
(1821, 13343)


In [24]:
# MultinomialNB
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()

x_train, x_test, y_train, y_test = train_test_split(X_tfidf_df, train_data['sentiment'].values,
                                                test_size=0.2, random_state=42)

model_mnb = mnb.fit(x_train,y_train)
y_pred_mnb = model_mnb.predict(x_test)
print('Accuracy %s' % accuracy_score(y_pred_mnb,y_test))
print(classification_report(y_test,y_pred_mnb))

Accuracy 0.7955202312138728
              precision    recall  f1-score   support

           0       0.85      0.70      0.77       671
           1       0.76      0.88      0.82       713

    accuracy                           0.80      1384
   macro avg       0.80      0.79      0.79      1384
weighted avg       0.80      0.80      0.79      1384



In [25]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(mnb, x_test, y_test, cv=10)
print("MultinominalNB score: ",scores.mean())

MultinominalNB score:  0.7247054530288813


In [26]:
# SVM
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from xgboost import XGBClassifier

svm = LinearSVC()
model_svm = svm.fit(x_train,y_train)
y_pred_svm = model_svm.predict(x_test)
print('Accuracy %s' % accuracy_score(y_pred_svm,y_test))
print(classification_report(y_test,y_pred_svm))

Accuracy 0.791907514450867
              precision    recall  f1-score   support

           0       0.81      0.75      0.78       671
           1       0.78      0.83      0.80       713

    accuracy                           0.79      1384
   macro avg       0.79      0.79      0.79      1384
weighted avg       0.79      0.79      0.79      1384



In [27]:
scores = cross_val_score(svm, x_test, y_test, cv=10)
print("SVM score:",scores.mean())

SVM score: 0.7348034615785632


In [28]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)
model_knn = knn.fit(x_train,y_train)
y_pred_knn = model_knn.predict(x_test)
print('Accuracy %s' % accuracy_score(y_pred_knn,y_test))
print(classification_report(y_test,y_pred_knn))

Accuracy 0.740606936416185
              precision    recall  f1-score   support

           0       0.75      0.71      0.73       671
           1       0.74      0.77      0.75       713

    accuracy                           0.74      1384
   macro avg       0.74      0.74      0.74      1384
weighted avg       0.74      0.74      0.74      1384



In [29]:
scores = cross_val_score(knn, x_test, y_test, cv=10)
print("KNN score:",scores.mean())

KNN score: 0.6675737670732979


In [30]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
model_dt = dt.fit(x_train,y_train)
y_pred_dt = model_dt.predict(x_test)
print('Accuracy %s' % accuracy_score(y_pred_dt,y_test))
print(classification_report(y_test,y_pred_dt))

Accuracy 0.6524566473988439
              precision    recall  f1-score   support

           0       0.65      0.62      0.63       671
           1       0.66      0.68      0.67       713

    accuracy                           0.65      1384
   macro avg       0.65      0.65      0.65      1384
weighted avg       0.65      0.65      0.65      1384



In [31]:
scores = cross_val_score(dt, x_test, y_test, cv=10)
print("Decision tree score:",scores.mean())

Decision tree score: 0.6098060681889271


In [32]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
model_rf = rf.fit(x_train,y_train)
y_pred_rf = model_rf.predict(x_test)
print('Accuracy %s' % accuracy_score(y_pred_rf,y_test))
print(classification_report(y_test,y_pred_rf))

Accuracy 0.7333815028901735
              precision    recall  f1-score   support

           0       0.77      0.64      0.70       671
           1       0.71      0.82      0.76       713

    accuracy                           0.73      1384
   macro avg       0.74      0.73      0.73      1384
weighted avg       0.74      0.73      0.73      1384



In [33]:
scores = cross_val_score(rf, x_test, y_test, cv=10)
print("Random forest score",scores.mean())

Random forest score 0.6711656761547284


In [34]:
# XGBoost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

y_train = y_train.astype(int)
y_test = y_test.astype(int)

xgboost_model = XGBClassifier()
model_xgb = xgboost_model.fit(x_train, y_train)

y_pred_xgb = model_xgb.predict(x_test)

print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


Accuracy: 0.7189306358381503
              precision    recall  f1-score   support

           0       0.77      0.60      0.67       671
           1       0.69      0.83      0.75       713

    accuracy                           0.72      1384
   macro avg       0.73      0.72      0.71      1384
weighted avg       0.73      0.72      0.71      1384



In [ ]:
scores = cross_val_score(xgboost_model, x_test, y_test, cv=10)
print("XGBoost score:",scores.mean())

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression


train_tokens = [clean(text) for text in train_data['text']]
w2v_model = Word2Vec(sentences=train_tokens, vector_size=100, window=5, min_count=1, workers=4)

def document_vector(doc):
    doc = [word for word in doc if word in w2v_model.wv.key_to_index]
    return np.mean(w2v_model.wv[doc], axis=0) if doc else np.zeros(w2v_model.vector_size)

X_train_w2v = np.array([document_vector(doc) for doc in train_tokens])
train_data['sentiment'] = train_data['sentiment'].astype(int)
y_train = train_data['sentiment'].values

classifier = LogisticRegression()
cv_results_w2v = cross_validate(classifier, X_train_w2v, y_train, cv=10, scoring=['accuracy', 'precision', 'recall', 'f1'])

print("Word2Vec Cross-Validation Results:")
print("Accuracy: ", np.mean(cv_results_w2v['test_accuracy']))
print("Precision: ", np.mean(cv_results_w2v['test_precision']))
print("Recall: ", np.mean(cv_results_w2v['test_recall']))
print("F1 Score: ", np.mean(cv_results_w2v['test_f1']))


In [ ]:
scores = cross_val_score(classifier, x_test, y_test, cv=10)
print("Word 2 Vec Score:",scores.mean())

In [ ]:
!pip install transformers torch

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
import numpy as np

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def bert_embedding(text):
    """Generate BERT embeddings for a given text."""
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**encoded_input)
    # Use the output of the [CLS] token (first token) for document classification
    return output.last_hidden_state[:, 0, :].numpy().squeeze()

# Obtain BERT embeddings for each document in the dataset
X_train_bert = np.array([bert_embedding(text) for text in train_data['text']])

# Logistic Regression model
log_reg = LogisticRegression(max_iter=1000)

# Perform 10-fold cross-validation
cv_results_bert = cross_validate(log_reg, X_train_bert, train_data['sentiment'].values, cv=10, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'])
print("BERT Cross-Validation Results:")
print("Accuracy: ", np.mean(cv_results_bert['test_accuracy']))
print("Precision: ", np.mean(cv_results_bert['test_precision_macro']))
print("Recall: ", np.mean(cv_results_bert['test_recall_macro']))
print("F1 Score: ", np.mean(cv_results_bert['test_f1_macro']))


## **Question 2 (20 Points)**

The purpose of the question is to practice different machine learning algorithms for **text clustering**.

Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

**Apply the listed clustering methods to the dataset:**
*   K-means
*   DBSCAN
*   Hierarchical clustering
*   Word2Vec
*   BERT

You can refer to of the codes from  the follwing link below.
https://www.kaggle.com/karthik3890/text-clustering

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import numpy as np
import torch
from collections import Counter

# Load the dataset
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')
df = df.dropna(subset=['Reviews'])

# Preprocess the text data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['Reviews'])

# K-Means Clustering
kmeans = KMeans(n_clusters=5, random_state=42)
labels_kmeans = kmeans.fit_predict(X)
print("K-Means Clustering:")
for i in range(5):
    cluster_index = np.where(labels_kmeans == i)
    sample_reviews = df.iloc[cluster_index]['Reviews'].sample(2, random_state=42).tolist()
    print(f"Cluster {i} examples: {sample_reviews}")

# DBSCAN Clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)
labels_dbscan = dbscan.fit_predict(X)
n_clusters_dbscan = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)
print("\nDBSCAN Clustering:")
print(f"Number of clusters: {n_clusters_dbscan}")
if n_clusters_dbscan > 1:
    for label in set(labels_dbscan):
        if label != -1:
            cluster_index = np.where(labels_dbscan == label)
            sample_reviews = df.iloc[cluster_index]['Reviews'].sample(1, random_state=42).tolist()
            print(f"Cluster {label} example: {sample_reviews}")

# Hierarchical Clustering
agg = AgglomerativeClustering(n_clusters=5)
labels_agg = agg.fit_predict(X.toarray())
print("\nHierarchical Clustering:")
for i in range(5):
    cluster_index = np.where(labels_agg == i)
    sample_reviews = df.iloc[cluster_index]['Reviews'].sample(2, random_state=42).tolist()
    print(f"Cluster {i} examples: {sample_reviews}")

# Word2Vec Clustering
sentences = df['Reviews'].apply(lambda x: x.split()).tolist()
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)
word_vectors = w2v_model.wv.vectors
kmeans_w2v = KMeans(n_clusters=5, random_state=42)
labels_w2v = kmeans_w2v.fit_predict(word_vectors)
print("\nWord2Vec Clustering:")
top_words = []
for i in range(5):
    cluster_index = np.where(labels_w2v == i)
    words = [w2v_model.wv.index_to_key[idx] for idx in cluster_index[0]]
    most_common = Counter(words).most_common(5)
    top_words.append(most_common)
    print(f"Cluster {i} most common words: {most_common}")

# BERT Clustering
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
bert_embeddings = []

for review in df['Reviews']:
    inputs = tokenizer(review, return_tensors='pt', truncation=True, max_length=128)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    bert_embeddings.append(embeddings.detach().numpy().squeeze())

bert_embeddings = np.array(bert_embeddings)
kmeans_bert = KMeans(n_clusters=5, random_state=42)
labels_bert = kmeans_bert.fit_predict(bert_embeddings)
print("\nBERT Clustering:")
for i in range(5):
    cluster_index = np.where(labels_bert == i)
    sample_reviews = df.iloc[cluster_index]['Reviews'].sample(2, random_state=42).tolist()
    print(f"Cluster {i} examples: {sample_reviews}")


In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
reviews= reviews[reviews['Reviews'].notnull()].head(5000)

reviews['punctuation'] = reviews['Reviews'].str.replace('[^\w\s].#','')#eliminatng punctuatiions
from nltk.corpus import stopwords
import nltk
stop = stopwords.words('english')
reviews['stopwords'] =reviews['punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))#eliminatng stopwords
reviews['numerics']=reviews['stopwords'].str.replace('[0-9]','')#eliminatng numercis
reviews['lowercase'] =reviews['numerics'].apply(lambda x: " ".join(x.lower() for x in x.split()))#to lowercase
from nltk.stem import PorterStemmer# for Stemming
stemmer = PorterStemmer()
reviews['stemming']=reviews['lowercase'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))
#Lemmatizing
from textblob import Word
reviews['cleaned_data'] = reviews['stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
reviews['cleaned_data']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer


vector = TfidfVectorizer()
tfidf = vector.fit_transform(reviews['cleaned_data'].values)

from sklearn.cluster import KMeans

kmodel = KMeans(n_clusters = 5,random_state=99) #Defining k means model
kmodel.fit(tfidf)
labels_kmeans = kmodel.labels_ #Creating labels
cluster_center_kmeans=kmodel.cluster_centers_ #Defining tcluster center
terms = vector.get_feature_names_out()
terms[1:10]
reviews['Label'] = kmodel.labels_
reviews

In [ ]:
#Finding top terms
print("Top terms:")
#Defing the centroid model
centroids = kmodel.cluster_centers_.argsort()[:, ::-1]
#Creating the loop for finding the top words
for i in range(5):
    print("Cluster %d:" % i, end='')
    for ind in centroids[i, :5]:
        print(' %s' % terms[ind], end='')
        print()

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

points = 2 * 100
def lower_b(nums, targ):
    l, r = 0, len(nums) - 1
    while l <= r:
        mid = int(l + (r - l) / 2)
        if nums[mid] >= targ:
            r = mid - 1
        else:
            l = mid + 1
    return l
def com_200th_near_tneigh(x, data):
    dists = []
    for val in data:
        dist = np.sum((x - val) **2 )
        if(len(dists) == 200 and dists[199] > dist):
            l = int(lower_b(dists, dist))
            if l < 200 and l >= 0 and dists[l] > dist:
                dists[l] = dist
            else:
              dists.append(dist)
              dists.sort()

    return dists[199]
import numpy as np
sent_vectors = [];
for sent in reviews['Reviews']:
    sent_vec = np.zeros(100)
    words =0;
    for word in sent:
        try:
            vec = tfidf.wv[word]
            sent_vec += vec
            words += 1
        except:
            pass
    sent_vec /= words
    sent_vectors.append(sent_vec)
sent_vectors = np.array(sent_vectors)
sent_vectors = np.nan_to_num(sent_vectors)
sent_vectors.shape
points = 2 * 100
model = DBSCAN(eps = 1, min_samples = points, n_jobs=-1)
model.fit(sent_vectors)
reviews['AVG Cluster Label'] = model.labels_
reviews

In [ ]:
for i in range(5):
    print("5 reviews of ensured to cluster ", i)
    print("-" * 70)
    print(reviews.iloc[reviews.groupby(['Label']).groups[i][0]]['cleaned_data'])
    print('\n')
    print(reviews.iloc[reviews.groupby(['Label']).groups[i][5]]['cleaned_data'])
    print('\n')
    print("_" * 70)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
#Defining model
cluster = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward')
Agg=cluster.fit_predict(sent_vectors)
#assigning Labels
aggdf = reviews
aggdf['AVG Cluster Label'] = cluster.labels_
aggdf.head()

In [ ]:
aggdf.groupby(['Label'])['cleaned_data'].count()

In [ ]:
for index in range(5):
    print("Review", index)
    print("-" * 100)
    print(aggdf.iloc[aggdf.groupby(['Label']).groups[index][0]]['cleaned_data'])
    print('\n')
    print(aggdf.iloc[aggdf.groupby(['Label']).groups[index][1]]['cleaned_data'])
    print('\n')
    print("_" * 100)

In [ ]:
import gensim

# Create a list of cleaned reviews
cleaned_reviews = reviews['cleaned_data'].tolist()

# Train a Word2Vec model
model = gensim.models.Word2Vec(cleaned_reviews, size=100, window=5, min_count=1, workers=4)

# Extract the embeddings for the reviews
review_vectors = []
for review in cleaned_reviews:
    vector = np.zeros(100)
    words = review.split()
    for i, word in enumerate(words):
        if word in model.wv.vocab:
            vector += model.wv[word]
    vector /= len(words)
    review_vectors.append(vector)

# Convert the list of vectors to a NumPy array
review_vectors = np.array(review_vectors)


In [ ]:
from transformers import BertTokenizer, BertModel

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Extract the embeddings for the reviews
review_vectors = []
for review in cleaned_reviews:
    inputs = tokenizer(review, return_tensors='pt')
    outputs = model(**inputs)
    vector = outputs.last_hidden_state[:, 0, :]
    review_vectors.append(vector.detach().numpy())

# Convert the list of vectors to a NumPy array
review_vectors = np.array(review_vectors)


**In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.**

**Write your response here:**
The computation is taking too long for all the algorithms. K-Means shows some distinct clusters since the data set is large. DBScan is inaccurate because of large data set. Hierarchical clustering is good but the score is less than k-means. Word 2 vec is good.

.

.

.

.

.




# Mandatory Question

**Important: Reflective Feedback on this exercise**

Please provide your thoughts and feedback on the exercises you completed in this assignment.


**(Your submission will not be graded if this question is left unanswered)**



In [ ]:
# Your answer here (no code for this question, write down your answer as detail as possible for the above questions):

'''
Please write you answer here:
It takes time to write a code like this. For me personally, I have another class
on thursday and almost very little time to complete the exercise. The compiling
of the codes take a lot of time. Couldn't learn much while doing this but will
look at it again after submitting it.

'''